# Proyek Analisis Data: E-Commerce Public Dataset
- **Nama:** Mukhlis Maulana Al-Fakih
- **Email:** mukhlis.maulanaalf@gmail.com
- **ID Dicoding:** alfakih

## Menentukan Pertanyaan Bisnis

- Produk atau kategori produk mana yang paling banyak terjual dan memiliki potensi peningkatan penjualan?
- Siapa pelanggan paling bernilai dan bagaimana karakteristik pelanggan yang melakukan pembelian berulang?

## Import Semua Packages/Library yang Digunakan

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

## Data Wrangling

### Gathering Data

In [15]:
customer_data = pd.read_csv('data/olist_customers_dataset.csv')
geo_data = pd.read_csv('data/olist_geolocation_dataset.csv')
order_items_data = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments_data = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews_data = pd.read_csv('data/olist_order_reviews_dataset.csv')
orders_data = pd.read_csv('data/olist_orders_dataset.csv')
products_data = pd.read_csv('data/olist_products_dataset.csv')
sellers_data = pd.read_csv('data/olist_sellers_dataset.csv')
product_category_data = pd.read_csv('data/product_category_name_translation.csv')


**Insight:**

<ul>
    <li><strong>Customer-Centric Analysis:</strong> Dataset ini menyediakan informasi yang cukup untuk melakukan
      analisis berbasis pelanggan, baik untuk melihat distribusi geografis pelanggan, frekuensi pembelian, hingga ulasan
      produk.</li>
    <li><strong>Product-Centric Analysis:</strong> Data produk dan pesanan dapat memberikan insight mengenai produk
      terlaris, kategori yang paling laris, dan produk dengan potensi untuk lebih dipromosikan.</li>
    <li><strong>Operational Analysis:</strong> Melalui data pengiriman, kita bisa menganalisis efektivitas logistik dan
      pengaruhnya terhadap kepuasan pelanggan.</li>
    <li><strong>Segmentasi Pelanggan:</strong> Dataset memungkinkan untuk melakukan segmentasi pelanggan berdasarkan
      perilaku pembelian (RFM) serta mengidentifikasi pelanggan bernilai tinggi untuk strategi pemasaran yang tepat.
    </li>
  </ul>


### Assessing Data

**Membuat fungsi untuk mempermudah proses assessing data**

In [23]:
def assess_data(path, date_columns=None):
    # 1. Memuat dataset dari path yang diberikan
    try:
        dataset = pd.read_csv(path)
        print(f"Dataset loaded successfully from {path}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return
    
    # 2. Menampilkan beberapa baris pertama
    print("\n--- First 5 Rows of Data ---")
    print(dataset.head())
    
    # 3. Menampilkan informasi struktur dataset
    print("\n--- Dataset Information ---")
    print(dataset.info())
    
    # 4. Memeriksa missing values
    print("\n--- Missing Values ---")
    missing_values = dataset.isnull().sum()
    print(missing_values[missing_values > 0])
    
    # 5. Memeriksa duplikasi
    duplicates = dataset.duplicated().sum()
    print(f"\n--- Number of Duplicate Rows: {duplicates}")
    
    # 6. Mengonversi kolom tanggal (jika ada)
    if date_columns:
        for col in date_columns:
            if col in dataset.columns:
                dataset[col] = pd.to_datetime(dataset[col], errors='coerce')
                print(f"Converted {col} to datetime format.")
    
    # 7. Menampilkan deskripsi statistik untuk data numerik
    print("\n--- Statistical Summary ---")
    print(dataset.describe())
    
    # 8. Menampilkan distribusi data untuk kolom kategorikal
    print("\n--- Value Counts for Categorical Columns ---")
    for col in dataset.select_dtypes(include='object').columns:
        print(f"\n{col} value counts:")
        print(dataset[col].value_counts().head(10))  # Menampilkan 10 nilai terbanyak
    
    return dataset

In [25]:
# Dataset order_items_data
order_items_data = assess_data('data/olist_order_items_dataset.csv', date_columns=['shipping_limit_date'])

Dataset loaded successfully from data/olist_order_items_dataset.csv

--- First 5 Rows of Data ---
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   
3  7634da152a4610f1595efa32f14722fc  9d7a1d34a5052409006425275ba1c2b4   
4  ac6c3623068f30de03045865e4e10089  df560393f3a51e74553ab94004ba5c87   

   shipping_limit_date   price  freight_value  
0  2017-09-19 09:45:35   58.90          13.29  
1  2017-05-03 11:05:13  239.90          19

In [26]:
# Dataset product_data
products_data = assess_data('data/olist_products_dataset.csv')

Dataset loaded successfully from data/olist_products_dataset.csv

--- First 5 Rows of Data ---
                         product_id  product_category_name  \
0  1e9e8ef04dbcff4541ed26657ea517e5             perfumaria   
1  3aa071139cb16b67ca9e5dea641aaa2f                  artes   
2  96bd76ec8810374ed1b65e291975717f          esporte_lazer   
3  cef67bcfe19066a932b7673e239eb23d                  bebes   
4  9dc1a7de274444849c219cff195d0b71  utilidades_domesticas   

   product_name_lenght  product_description_lenght  product_photos_qty  \
0                 40.0                       287.0                 1.0   
1                 44.0                       276.0                 1.0   
2                 46.0                       250.0                 1.0   
3                 27.0                       261.0                 1.0   
4                 37.0                       402.0                 4.0   

   product_weight_g  product_length_cm  product_height_cm  product_width_cm  
0        

In [27]:
# Dataset customer_data
customer_data = assess_data('data/olist_customers_dataset.csv')

Dataset loaded successfully from data/olist_customers_dataset.csv

--- First 5 Rows of Data ---
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  

--- Dataset Information ---
<class 'pandas.core.frame.Da

In [28]:
# Dataset orders_data
orders_data = assess_data('data/olist_orders_dataset.csv')

Dataset loaded successfully from data/olist_orders_dataset.csv

--- First 5 Rows of Data ---
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-

In [29]:
# Dataset product_category_data
product_category_data = assess_data('data/product_category_name_translation.csv')

Dataset loaded successfully from data/product_category_name_translation.csv

--- First 5 Rows of Data ---
    product_category_name product_category_name_english
0            beleza_saude                 health_beauty
1  informatica_acessorios         computers_accessories
2              automotivo                          auto
3         cama_mesa_banho                bed_bath_table
4        moveis_decoracao               furniture_decor

--- Dataset Information ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None

--- Missing Values ---
Series([], dtype: int64)

--- Number of Duplicate Rows: 0

--- Statistical Summary ---
       product_category_name produc

**Insight:**
    <p><strong><code>order_items_data</code></strong>:</p>
    <ul>
      <li>Missing values: <strong>0</strong> (Tidak ada nilai yang hilang).</li>
      <li>Duplicate data: <strong>0</strong> (Tidak ada data yang terduplikasi).</li>
    </ul>
    <p><strong><code>products_data</code></strong>:</p>
    <ul>
      <li>Missing values:<ul>
          <li><code>product_category_name</code>: <strong>610</strong> missing values.</li>
          <li><code>product_name_lenght</code>: <strong>610</strong> missing values.</li>
          <li><code>product_description_lenght</code>: <strong>610</strong> missing values.</li>
          <li><code>product_photos_qty</code>: <strong>610</strong> missing values.</li>
          <li><code>product_weight_g</code>: <strong>2</strong> missing values.</li>
          <li><code>product_length_cm</code>: <strong>2</strong> missing values.</li>
          <li><code>product_height_cm</code>: <strong>2</strong> missing values.</li>
          <li><code>product_width_cm</code>: <strong>2</strong> missing values.</li>
        </ul>
      </li>
      <li>Duplicate data: <strong>0</strong> (Tidak ada data yang terduplikasi).</li>
    </ul>
    <p><strong><code>customer_data</code></strong>:</p>
    <ul>
      <li>Missing values: <strong>0</strong> (Tidak ada nilai yang hilang).</li>
      <li>Duplicate data: <strong>0</strong> (Tidak ada data yang terduplikasi).</li>
    </ul>
    <p><strong><code>orders_data</code></strong>:</p>
    <ul>
      <li>Missing values:<ul>
          <li><code>order_approved_at</code>: <strong>160</strong> missing values.</li>
          <li><code>order_delivered_carrier_date</code>: <strong>1783</strong> missing values.</li>
          <li><code>order_delivered_customer_date</code>: <strong>2965</strong> missing values.</li>
        </ul>
      </li>
      <li>Duplicate data: <strong>0</strong> (Tidak ada data yang terduplikasi).</li>
    </ul>
    <p><strong><code>product_category_data</code></strong>:</p>
    <ul>
      <li>Missing values: <strong>0</strong> (Tidak ada nilai yang hilang).</li>
      <li>Duplicate data: <strong>0</strong> (Tidak ada data yang terduplikasi).</li>
    </ul>
    <h3><strong>Kesimpulan</strong></h3>
    <p>
    Melihat jumlah missing values yang relatif kecil dibandingkan dengan ukuran dataset secara keseluruhan, sehingga tidak akan terlalu berdampak signifikan pada analisis. Maka proses cleaning data nanti akan dilakukan dengan cara menghapus data yang memiliki missing value tanpa mengganti nilainya.
    </p>

### Cleaning Data

In [32]:
# Dataset products_data
products_data_cleaned = products_data.dropna()

# Dataset orders_data
orders_data_cleaned = orders_data.dropna()

**Insight:**
- xxx
- xxx

## Exploratory Data Analysis (EDA)

### Explore ...

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2